# KZG Implementation(Python Version)

The KZG Commitment Scheme is a commitment scheme that allows to commit to a polynomial $\Phi(x) = \phi_0 +\phi_1x+\phi_2x^2+...+\phi_lx^l$, where $\Phi(x) \in F_p[x]$ . 'to commit' means proving that you know the polynomial $\Phi(x)$ without revealing it.

The KZG commitment scheme consists of 4 steps:
1. Setup
2. Commit to Polynomials
3. Prove an Evaluation
4. Verify an Evaluation Proof

## Step 0: Curve Preparation

Curve we used is BLS12-381
- Library we used: BLS21-381 curve implemented by Arkwork, using Python
- Lib doc: https://pypi.org/project/py-arkworks-bls12381/

Basic Information about BLS12-381:
- the first curve $G_1$ is defined over $F_p$ and statisfy $y^2 = x^3 + 4$: $$E_1(F_p) = \{(x,y): x, y \in F_p\ satisfy \  y^2 = x^3 + 4\} \cup \{\vartheta\}$$
    - $p$(381 bits) = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
    - #$E_1(F_p) = h_1*n$ 
        - h_1 = 0x396c8c005555e1568c00aaab0000aaab
        - n = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
    - Generator of $G_1$:
        - x = 0x17f1d3a73197d7942695638c4fa9ac0fc3688c4f9774b905a14e3a3f171bac586c55e83ff97a1aeffb3af00adb22c6bb
        - y = 0x08b3f481e3aaa0f1a09e30ed741d8ae4fcf5e095d5d00af600db18cb2c04b3edd03cc744a2888ae40caa232946c5e7e1
- the second curve $G_2$ is defined over $F_{p^2}$ and statisfy $y^2 = x^3 + 4(u+1)$: $$E_2(F_{p^2}) = \{(x,y): x, y \in F_{p^2}\ satisfy \  y^2 = x^3 + 4(u+1)\} \cup \{\vartheta\}$$
    - #$E_2(F_{p^2}) = h_2*n$ 
        - h_2 = 0x5d543a95414e7f1091d50792876a202cd91de4547085abaa68a205b2e5a7ddfa628f1cb4d9e82ef21537e293a6691ae1616ec6e786f0c70cf1c38e31c7238e5
        - n = 0x73eda753299d7d483339d80809a1d80553bda402fffe5bfeffffffff00000001
    - Generator of $G_1$:
        - x = 0x024AA2B2F08F0A91260805272DC51051C6E47AD4FA403B02B4510B647AE3D1770BAC0326A805BBEFD48056C8C121BDB8 + 0x13E02B6052719F607DACD3A088274F65596BD0D09920B61AB5DA61BBDC7F5049334CF11213945D57E5AC7D055D042B7E u
        - y = 0x0CE5D527727D6E118CC9CDC6DA2E351AADFD9BAA8CBDD3A76D429A695160D12C923AC9CC3BACA289E193548608B82801 + 0x0606C4A02EA734CC32ACD2B02BC28B99CB3E287E85A763AF267492AB572E99AB3F370D275CEC1DA1AAA9075FF05F79BE u

        

In [1]:
pip install py-arkworks-bls12381


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 1: Setup

The first step is an one-time trusted setup and once it has done once, the following steps can be done repeatedly
1. Let $G_1$ and $G_2$ be pairing-friendly elliptic curve groups
2. Let $g_1$ be a generator of $G_1$ and $g_2$ be a generator of $G_2$
3. Let $l$ be the maximum degree of the polymonials we want to commit to ($l < p$)
4. Pick a random field element as secret parameter $\tau \in F_p$(usually done by MPC, to simplify, we just randomly choose one here)
5. Compute pp(public parameters)$$pk = (g_1, g_1^\tau, g_1^{\tau^2},...,g_1^{\tau^l}), vk = g_2^\tau$$ and release it publicly
6. Discard secret parameter $\tau$ once the setup ceremony is done so that nobody can figure out its value

In [2]:
from py_arkworks_bls12381 import G1Point, G2Point, Scalar
from sage import *

# 1. Let G be a pairing-friendly elliptic curve group, and G1 is the G

# 2. Let g be a generator of G
g1 = G1Point()
g2 = G2Point()

# 3. Let l be the maximum degree of the polymonials, which is 12(by the definition of the curve)
l = 12

# 4. Pick a random field element as secret parameter t
# ! cannot use real p, when exceed 2^127, scalar will warning "overflow"
# p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
# F_p = GF(p)
# t = F_p.random_element()
p = 115249
F_p = GF(p)
t = F_p.random_element()
print(f'secret parameter 𝜏 is {t}')

# 5. Compute pp(public parameters)
def compute_public_parameters(g1, g2, t, l):
    
    pk = []
    accumulated = Scalar(1)
    t_scalar = Scalar(t)
    for i in range(l + 1):
        pk.append(g1 * accumulated)
        accumulated = accumulated * t_scalar
    vk = g2 * Scalar(t)
    return pk, vk

pk, vk = compute_public_parameters(g1, g2, t, l)
print(f'pk: {pk}, vk: {vk}')

secret parameter 𝜏 is 113055
pk: [<builtins.G1Point object at 0x11321c660>, <builtins.G1Point object at 0x11321c5b0>, <builtins.G1Point object at 0x11321c710>, <builtins.G1Point object at 0x11321c7c0>, <builtins.G1Point object at 0x11321c870>, <builtins.G1Point object at 0x11321c920>, <builtins.G1Point object at 0x11321c9d0>, <builtins.G1Point object at 0x11321ca80>, <builtins.G1Point object at 0x11321cb30>, <builtins.G1Point object at 0x11321cbe0>, <builtins.G1Point object at 0x11321cc90>, <builtins.G1Point object at 0x11321cd40>, <builtins.G1Point object at 0x11321cdf0>], vk: 955aaec434ed56593cf08a87c95b9693e6e1466bb819a882ee6b9452dba14f80b93aa7b96a2c2c7143fe5af4ad0d597104b857e281154b3675d49db5af50cdbb81c6fda00c47cdb4169aad1487184c3c2a9e4fb66bb822f13daa080190b7b30a


## Step 2: Commit to Polynomials
In reality, we arithmetize circuits and use Plonkish to get polynomials in this step
1. Given a polynomial $\Phi(x) = \sum_{i=0}^l \phi_i x^i$
2. Compute and output commitment $c = g^{\Phi(\tau)}$
   - Wait! $\tau$ has already been discard right? How can committer compute $\tau$?
   - Although he cannot compute $\Phi(\tau)$ directly, he can use public parameters to help with it: 
$$\prod_{i=0}^l(g^{\tau^i})^{\phi_i} = g^{\sum_{i=0}^l \phi_i \tau^i} = g^{\Phi(\tau)}$$

In [3]:
# assume we have done plonk and get our polynomial Φ(𝑥)=4x^2+5x+3 and some point (0,3),(1,12),(3,54) on the poly
# poly_coefs = [3, 5, 4]
poly_point = [(0,3),(1,12),(3,54)]

In [4]:
# create a polynomial ring F_p(x)
F_p_x.<x> = PolynomialRing(F_p)
F_p_x

Univariate Polynomial Ring in x over Finite Field of size 115249

In [5]:
poly = F_p_x.lagrange_polynomial(poly_point)
poly

4*x^2 + 5*x + 3

In [6]:
def poly_commitment(pk, g1, poly):

    poly_coefs = poly.coefficients()
    com = G1Point.identity()
    
    for i in range(len(poly_coefs)):
        com = pk[i] * Scalar(poly_coefs[i]) + com
    return com
        
com = poly_commitment(pk, g1, poly)
print(f'commitment of polynomial is {com}')

commitment of polynomial is 984faa11a11d8a551db688a58f14d79bb7ba99c26a92910d9cc73eaa4806bd014ed2dbcf5ff1c950eebdd2961f6a21e6


## Step 3: Prove an Evaluation
In this period, the Verifier will ask Prover to 'OPEN' the commitment $c$ to a random specific point $a \in F_p$, in other word, Prover have to evaluation $\Phi(x)$ and commit the result in the form of opening triplet $OT = (a, b, \pi)$
1. Given an evaluation $\Phi(a) = b$
2. Compute and output proof of the evaluation $\pi = g^{q(\tau)}$, where $q(x) := \frac{\Phi(x)-b}{x-a}$
    - $q(x)$ is quotient polynomial: if $\Phi(a) = b$, that means $a$ is a root of $\Phi(x) - b$
    - so $\Phi(x) - b$ can be expressed as $\Phi(x) - b = q(x)(x-a)$, $q(x)$ is a polynomial
    - on the other hand, $q(x)$ exists if and only if $\Phi(a) = b$, so the existence of this quotient polynomial therefore serves as a proof of the evaluation

In [7]:
# Verifier first choose the random point a
# a = F_p.random_element()
# b = poly(a)
a = 1
b = poly(a)

print(f'random element we chosen is {a}, the evaluation b is {b}')

# compute q(x)
q_poly = ((poly-b)/(x-a)).numerator()
print(f'quotient polynomial is {q_poly}')

# compute proof of the evaluation pi
pi = poly_commitment(pk, g1,  q_poly)
print(f'proof of the evaluation pi is {pi}')

random element we chosen is 1, the evaluation b is 12
quotient polynomial is 4*x + 9
proof of the evaluation pi is 8a2f6557e132551b240f7c1e14796119067305e5f45744145ab376f1638f9043526f6f0bbd47005ac2ad5839a6785bbf


## Step 4: Verify an Evaluation Proof
1. Given a commitment $c = g^{\Phi(\tau)}$, and an evaluation $\Phi(a) = b$, and a proof $\pi = g^{q(\tau)}$
2. Verify that $e(\frac{c}{g^b}, g) = e(\pi,\frac{g^\tau}{g^a})$, where $e$ is a non-trivial bilinear mapping
    - the purpose of verification: $\Phi(x) - b = q(x)(x-a)$, checking this equality holds for $x = \tau$
    - according to the definition of bilinear mapping, it is equivalent to: $e(g_1, g_2)^{\Phi(\tau) - b} = e(g_1, g_2)^{q(\tau)(\tau-a)}$, that is $e(g_1^{\Phi(\tau) - b}, g_2) = e(g_1^{q(\tau)}, g_2^{\tau-a})$
    - that is $e(com-g_1^b, g_2) = e(\pi, vk - g_2^a)$

In [9]:
from py_arkworks_bls12381 import GT


# now it is time to do the varification
assert GT.pairing(com - g1*Scalar(b), g2) == GT.pairing(pi, vk - g2*Scalar(a))

In [10]:
GT.pairing(com - g1*Scalar(b), g2)

In [11]:
GT.pairing(pi, vk - g2*Scalar(a))

## Reference
1. https://ethresear.ch/t/yet-another-curve-but-the-curve-for-your-kzg/12861
2. https://scroll.io/blog/kzg
3. https://dankradfeist.de/ethereum/2020/06/16/kate-polynomial-commitments.html
4. https://alinush.github.io/2020/05/06/kzg-polynomial-commitments.html